In [47]:
import pandas as pd
import numpy as np
import sqlalchemy
pd.set_option('display.max.columns',None)
pd.options.display.max_colwidth = 100

In [48]:
# Uploading Edinburgh bikes data file.
edbikes_df = pd.read_csv('edinburgh_bikes_202205172048.csv',delimiter=';', decimal=',',index_col=0)
edbikes_df.head()

,started_at,ended_at,duration,start_station_id,start_station_name,start_station_description,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_description,end_station_latitude,end_station_longitude
index,,,,,,,,,,,,,
0,2018-09-15 08:52:05,2018-09-15 09:11:48,1182,247,Charlotte Square,North Corner of Charlotte Square,55.95233546161639,-3.207101172107286,259,St Andrew Square,North East corner,55.95472778937899,-3.192652969253117
1,2018-09-15 09:24:33,2018-09-15 09:41:09,995,259,St Andrew Square,North East corner,55.95474881217642,-3.192773668658787,262,Canonmills,near Tesco's,55.96280408759764,-3.196283585062929
2,2018-09-15 09:48:54,2018-09-15 10:46:40,3466,262,Canonmills,near Tesco's,55.96280408759764,-3.196283585062929,250,Victoria Quay,Entrance to Scottish Government Office,55.977637787726984,-3.174115590621568
3,2018-09-16 12:01:36,2018-09-16 12:25:26,1430,255,Kings Buildings 4,X-Y Cafe,55.92200087060626,-3.1769018141649212,254,Kings Building 3,Kings Building House,55.923478807846635,-3.175384584907078
4,2018-09-16 12:03:43,2018-09-16 12:11:16,452,255,Kings Buildings 4,X-Y Cafe,55.92200087060626,-3.1769018141649212,253,Kings Building 2,Sanderson Building,55.92320215516664,-3.17164624536349


In [5]:
# Data exploration: time frame. Table contains Edinburgh bikes borrowings from September 15th, 2018 until end June 2021.

edbikes_df[['started_at','ended_at']].sort_values('started_at')

,started_at,ended_at
index,,
0,2018-09-15 08:52:05,2018-09-15 09:11:48
1,2018-09-15 09:24:33,2018-09-15 09:41:09
2,2018-09-15 09:48:54,2018-09-15 10:46:40
3,2018-09-16 12:01:36,2018-09-16 12:25:26
4,2018-09-16 12:03:43,2018-09-16 12:11:16
...,...,...
12636,2021-06-30 23:30:31,2021-07-01 00:06:10
12637,2021-06-30 23:36:16,2021-07-01 00:05:40
12638,2021-06-30 23:49:03,2021-07-01 00:11:25


In [49]:
# Start stations. There are 198 stations at least once used for start.
start = (edbikes_df[['start_station_id','start_station_name']].drop_duplicates(subset='start_station_id')
.rename(columns={'start_station_id':'station_id','start_station_name':'station_name'})
)
print('Start stations:')
print('\n',start)

Start stations:

        station_id                                               station_name
index                                                                       
0             247                                           Charlotte Square
1             259                                           St Andrew Square
2             262                                                 Canonmills
3             255                                          Kings Buildings 4
5             253                                           Kings Building 2
6             252                                          Kings Buildings 1
7             254                                           Kings Building 3
9             264                                              Pollock Halls
11            246                                    Royal Commonwealth Pool
12            265                                               Meadows East
13            183                                         

In [50]:
# End stations. There are 199 stations at least once used to end the borrowing.
end = (edbikes_df[['end_station_id','end_station_name']].drop_duplicates(subset='end_station_id')
       .rename(columns={'end_station_id':'station_id','end_station_name':'station_name'})
      )
print("End stations:")
print('\n',end)

End stations:

        station_id                                               station_name
index                                                                       
0             259                                           St Andrew Square
1             262                                                 Canonmills
2             250                                              Victoria Quay
3             254                                           Kings Building 3
4             253                                           Kings Building 2
5             252                                          Kings Buildings 1
7             264                                              Pollock Halls
10            246                                    Royal Commonwealth Pool
11            265                                               Meadows East
12            248                                              Bristo Square
13            258                                           

In [51]:
# All stations. There are 200 stations all together at least once used to start or at least once to end.
stations = pd.concat([end,start]).drop_duplicates().sort_values('station_id')
print("All stations at least once used to start or once to end:")
print('\n',stations)

All stations at least once used to start or once to end:

        station_id                                               station_name
index                                                                       
27            171                                              George Square
48            183                                            Waverley Bridge
14            189                                              City Chambers
135           225                                             Waverley Court
1278          241                                                      Depot
1126          242                                              Virtual Depot
10            246                                    Royal Commonwealth Pool
28            247                                           Charlotte Square
12            248                                              Bristo Square
66            249                                             Fountainbridge
2             250

According to stations names some of them should be temporary: Launch Day Event, Royal Highland Show - East Gate (19th to 23rd June), Royal Highland Show - West Gate (19th to 23rd June), 
Ingliston Park and Ride (19th to 23rd June), Meadows - Edinburgh Climate Festival, Holyrood Park - Woman's Tour Of Scotland (Event 11/08/19), Pleasance - Edinburgh University Sports Fair,
HSBC UK Lets Ride - Meadows Event, Cycling Scotland Conference.



In [6]:
# Temporary stations time frame.
(edbikes_df.query("start_station_id in (267,980,981,982,991,1032,1057,1094,1740) or end_station_id in (267,980,981,982,991,1032,1057,1094,1740)")
 .sort_values('start_station_id')
)

,started_at,ended_at,duration,start_station_id,start_station_name,start_station_description,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_description,end_station_latitude,end_station_longitude
index,,,,,,,,,,,,,
42,2018-09-17 09:57:10,2018-09-17 10:11:39,869,189,City Chambers,City Chambers Quadrangle,55.95010933017899,-3.190257770093808,267,Launch Day Event,The Meadows,55.942551339390945,-3.191380756192416
29,2018-09-17 09:41:25,2018-09-17 10:05:46,1461,189,City Chambers,City Chambers Quadrangle,55.95010933017899,-3.190257770093808,267,Launch Day Event,The Meadows,55.942551339390945,-3.191380756192416
31,2018-09-17 09:46:36,2018-09-17 10:06:08,1171,189,City Chambers,City Chambers Quadrangle,55.95010933017899,-3.190257770093808,267,Launch Day Event,The Meadows,55.942551339390945,-3.191380756192416
32,2018-09-17 09:47:06,2018-09-17 09:58:24,677,189,City Chambers,City Chambers Quadrangle,55.95010933017899,-3.190257770093808,267,Launch Day Event,The Meadows,55.942551339390945,-3.191380756192416
34,2018-09-17 09:47:31,2018-09-17 10:07:39,1208,189,City Chambers,City Chambers Quadrangle,55.95010933017899,-3.190257770093808,267,Launch Day Event,The Meadows,55.942551339390945,-3.191380756192416
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10297,2019-09-22 10:40:45,2019-09-22 11:15:32,2086,1094,HSBC UK Lets Ride - Meadows Event,20 point virtual docking station for UK lets Ride Event,55.939977758149666,-3.189862157672223,1094,HSBC UK Lets Ride - Meadows Event,20 point virtual docking station for UK lets Ride Event,55.939977758149666,-3.189862157672223
10262,2019-09-22 10:11:54,2019-09-22 10:58:36,2801,1094,HSBC UK Lets Ride - Meadows Event,20 point virtual docking station for UK lets Ride Event,55.939977758149666,-3.189862157672223,1094,HSBC UK Lets Ride - Meadows Event,20 point virtual docking station for UK lets Ride Event,55.939977758149666,-3.189862157672223
873,2019-11-04 16:04:18,2019-11-04 16:19:15,897,1102,Haymarket Station,Haymarket Station,55.94556945067266,-3.21818468327956,1740,Cycling Scotland Conference,Murrayfield Stadium,55.940886347739024,-3.240777565829148


Temporary stations were used in 2018 and 2019.

In [28]:
# We need to decide how to identify Active stations.
# Active stations  will be all stations (Start stations and End stations) used last year from 30.6.2020 until the last record.


In [42]:
# Start stations used last year.
start_active = (edbikes_df.query("started_at >= '2020-06-30' or ended_at >= '2020-06-30'")[['start_station_id','start_station_name']]
 .drop_duplicates(subset='start_station_id').rename(columns={'start_station_id':'station_id','start_station_name':'station_name'})
)

In [43]:
# End stations used last year.
end_active = (edbikes_df.query("started_at >= '2020-06-30' or ended_at >= '2020-06-30'")[['end_station_id','end_station_name']]
 .drop_duplicates(subset='end_station_id').rename(columns={'end_station_id':'station_id','end_station_name':'station_name'})
)

In [44]:
# All stations used last year: Active stations.
active_stations = pd.concat([end_active,start_active]).drop_duplicates().sort_values('station_id')
active_stations.values[:,0]

array([171, 183, 189, 225, 246, 247, 248, 249, 250, 251, 252, 253, 254,
       256, 258, 259, 260, 262, 264, 265, 285, 289, 296, 299, 349, 358,
       1017, 1019, 1024, 1025, 1028, 1030, 1031, 1038, 1039, 1042, 1050,
       1051, 1052, 1055, 1090, 1091, 1092, 1093, 1096, 1097, 1098, 1102,
       1720, 1721, 1722, 1725, 1726, 1727, 1728, 1729, 1730, 1731, 1737,
       1738, 1739, 1743, 1744, 1745, 1747, 1748, 1749, 1752, 1753, 1754,
       1756, 1757, 1758, 1763, 1764, 1765, 1766, 1767, 1768, 1769, 1770,
       1798, 1799, 1800, 1807, 1808, 1809, 1813, 1814, 1815, 1818, 1819,
       1820, 1821, 1822, 1823, 1824, 1859, 1860, 1864, 1865, 1866, 1868,
       1869, 1870, 1871, 1874, 1877, 2259, 2263, 2265, 2268], dtype=object)

In [52]:
# Inactive stations: all stations not used since June 30, 2020.
inactive_stations = pd.concat([stations['station_id'],active_stations['station_id'],active_stations['station_id']]).drop_duplicates(keep=False)
inactive_stations.values

array([ 241,  242,  255,  257,  261,  266,  267,  273,  275,  277,  280,
        284,  290,  297,  340,  341,  342,  343,  344,  345,  346,  347,
        350,  351,  352,  353,  354,  355,  356,  357,  359,  365,  366,
        648,  820,  860,  862,  863,  864,  865,  866,  867,  868,  869,
        870,  871,  872,  873,  874,  875,  876,  877,  878,  879,  880,
        881,  882,  883,  884,  885,  887,  888,  889,  890,  891,  901,
        964,  965,  980,  981,  982,  991, 1018, 1026, 1027, 1032, 1033,
       1040, 1041, 1056, 1057, 1094, 1095, 1723, 1724, 1740, 1746, 1857],
      dtype=int64)

In [3]:
# The most frequented stations will be stations with the highest sum of starts and ends.

In [17]:
# Dataframes with sum of starts and sum of ends.
df = edbikes_df.assign(cnt=0).groupby(['start_station_id','started_at']).count()[['cnt']]
df1 = df.groupby('start_station_id').sum().rename(columns={'cnt':'sum_borrowings'}).rename_axis('station_id')
df2 = edbikes_df.assign(cnt=0).groupby(['end_station_id','ended_at']).count()[['cnt']]
df3 = df2.groupby('end_station_id').sum().rename(columns={'cnt':'sum_returns'}).rename_axis('station_id')

In [56]:
# Stations by frequency.
activity_df = df3.join(df1)
activity_df.fillna(value=0,axis=1,inplace=True)
activity_df['frequency']=activity_df.sum_returns+activity_df.sum_borrowings
activity_df.sort_values('frequency',ascending=False)

,sum_returns,sum_borrowings,frequency
station_id,,,
265,15225,17390.0,32615.0
1728,16656,11843.0,28499.0
250,15254,10662.0,25916.0
262,11940,9551.0,21491.0
248,7898,12846.0,20744.0
259,8372,12218.0,20590.0
1024,8792,11373.0,20165.0
358,11052,7964.0,19016.0
1824,11264,7288.0,18552.0


In [3]:
# Number of days in records edbikes_df.
days=(pd.to_datetime(edbikes_df['started_at'],format="%Y/%m/%d").dt.date.max())-(pd.to_datetime(edbikes_df['started_at'],format="%Y/%m/%d").dt.date.min())
days

datetime.timedelta(days=1019)

In [13]:
# 10 most frequent stations.
activity_df['day_frequency']= (activity_df.frequency/1019).astype(int)
(
 activity_df.merge(stations,on='station_id',how='inner')[['station_id','station_name','frequency','day_frequency']]
 .sort_values('frequency',ascending=False).head(10)
)

,station_id,station_name,frequency,day_frequency
24,265,Meadows East,32615.0,32
139,1728,Portobello - Kings Road,28499.0,27
10,250,Victoria Quay,25916.0,25
22,262,Canonmills,21491.0,21
8,248,Bristo Square,20744.0,20
19,259,St Andrew Square,20590.0,20
101,1024,Meadow Place 2,20165.0,19
55,358,Leith Walk,19016.0,18
183,1824,Duke Street,18552.0,18
1,183,Waverley Bridge,16971.0,16
